In [1]:
import numpy as np
import open3d as o3d

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import pandas as pd
import numpy as np

import time

In [2]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, hidden_size)
        self.fc5 = nn.Linear(hidden_size, hidden_size)
        self.fc6 = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.relu(self.fc4(x))
        x = self.relu(self.fc5(x))
        x = self.fc6(x)
        return x

In [7]:
hand_lmks_file = np.load('../hand_landmarks.npz')

In [4]:
hand_lmks_input, hand_lmks_gt = hand_lmks_file["landmarks_input"], hand_lmks_file["landmarks_output"]

In [5]:
assert hand_lmks_input.shape[0] == hand_lmks_gt.shape[0]

(69, 21, 3)

In [5]:
input_cam1_cols = ["cam1_X00_in", "cam1_Y00_in", "cam1_Z00_in",
                   "cam1_X01_in", "cam1_Y01_in", "cam1_Z01_in",
                   "cam1_X02_in", "cam1_Y02_in", "cam1_Z02_in",
                   "cam1_X03_in", "cam1_Y03_in", "cam1_Z03_in",
                   "cam1_X04_in", "cam1_Y04_in", "cam1_Z04_in",
                   "cam1_X05_in", "cam1_Y05_in", "cam1_Z05_in",
                   "cam1_X06_in", "cam1_Y06_in", "cam1_Z06_in",
                   "cam1_X07_in", "cam1_Y07_in", "cam1_Z07_in",
                   "cam1_X08_in", "cam1_Y08_in", "cam1_Z08_in",
                   "cam1_X09_in", "cam1_Y09_in", "cam1_Z09_in",
                   "cam1_X10_in", "cam1_Y10_in", "cam1_Z10_in",
                   "cam1_X11_in", "cam1_Y11_in", "cam1_Z11_in",
                   "cam1_X12_in", "cam1_Y12_in", "cam1_Z12_in",
                   "cam1_X13_in", "cam1_Y13_in", "cam1_Z13_in",
                   "cam1_X14_in", "cam1_Y14_in", "cam1_Z14_in",
                   "cam1_X15_in", "cam1_Y15_in", "cam1_Z15_in",
                   "cam1_X16_in", "cam1_Y16_in", "cam1_Z16_in",
                   "cam1_X17_in", "cam1_Y17_in", "cam1_Z17_in",
                   "cam1_X18_in", "cam1_Y18_in", "cam1_Z18_in",
                   "cam1_X19_in", "cam1_Y19_in", "cam1_Z19_in",
                   "cam1_X20_in", "cam1_Y20_in", "cam1_Z20_in"]
input_cam1_header = ','.join(input_cam1_cols)

In [6]:
#np.savetxt('hand_landmark_data.csv', data, delimiter=',', fmt='%f', header=output_cols_header, comments='')

# Create a pseudo code for storing data to .csv file

In [7]:
def write_data_to_csv(file_name, data, num_cam=2):
    num_points_each_joint = 3
    num_joints_each_hand = 21
    num_input_cols = num_cam * num_points_each_joint * num_joints_each_hand

    input_header = input_cam1_header
    for i in range(2, num_cam+1):
        input_cam_i_header = input_cam1_header.replace("cam1", "cam{}".format(i))
        input_header += ',' + input_cam_i_header

    output_header = input_cam1_header.replace("cam1_", "").replace("in", "out")
    csv_header = input_header + ',' + output_header

    assert len(csv_header.split(",")) == data.shape[1]

    np.savetxt(file_name, data, delimiter=',', fmt='%f', header=csv_header, comments='')

In [8]:
num_frames = hand_lmks.shape[0]
num_input_cols = 2 * 3 * 21  # num_cameras * num_points_each_landmark * num_landmarks_each_hand

input_fake_data = np.random.rand(num_frames, num_input_cols)

In [9]:
output_data = hand_lmks.reshape(hand_lmks.shape[0], -1)

In [10]:
total_data = np.concatenate([input_fake_data, output_data], axis=1)

In [11]:
write_data_to_csv("hand_landmark_data.csv", total_data)

# Prepare data and train model

In [8]:
hand_lmks_file["landmarks"]

array([[[ 5.68434189e-14, -5.68434189e-14,  0.00000000e+00],
        [-1.52848820e+00,  9.56189611e+00,  1.67466843e+01],
        [-5.79381980e+00,  2.91853585e+01,  2.77939183e+01],
        ...,
        [-1.54552673e+00,  7.99153625e+01, -1.68866025e+01],
        [-3.03854417e+00,  9.23897190e+01, -1.86039410e+01],
        [-3.62491441e+00,  1.02715554e+02, -1.96881872e+01]],

       [[ 2.84217094e-14, -5.68434189e-14,  0.00000000e+00],
        [-1.10309858e+00,  8.05256239e+00,  1.66651296e+01],
        [-3.95009101e+00,  2.78328438e+01,  2.85878426e+01],
        ...,
        [-1.62421015e+00,  8.02314955e+01, -1.60939791e+01],
        [-3.47709417e+00,  9.21763154e+01, -1.78078697e+01],
        [-3.27198144e+00,  1.02729177e+02, -1.80885201e+01]],

       [[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [-2.27150639e-02,  8.12268579e+00,  1.72475406e+01],
        [-2.22002063e+00,  2.74942519e+01,  2.90146209e+01],
        ...,
        [-2.53066685e+00,  8.06382105e+01,

In [6]:
num_output_nodes = output_data.shape[1]

NameError: name 'output_data' is not defined

In [5]:
# Load the data from a CSV file
data = pd.read_csv('hand_landmark_data.csv')

X = data.iloc[:, :-(num_output_nodes)]
Y = data.iloc[:, -(num_output_nodes):]

"""
# Convert the data to PyTorch tensors
X_train = torch.tensor(X, dtype=torch.float32)
Y_train = torch.tensor(Y, dtype=torch.float32)

# Create DataLoader
train_dataset = TensorDataset(X_train, Y_train)
test_dataset = TensorDataset(X_train, Y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)"""

NameError: name 'num_output_nodes' is not defined

In [46]:
input_size = X.shape[1]
hidden_size = 64
output_size = num_output_nodes  # Regression output

model = MLP(input_size, hidden_size, output_size)
criterion = nn.MSELoss()  # Mean Squared Error Loss
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [48]:
num_epochs = 10000

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    if epoch % 500 == 0 or epoch == num_epochs - 1:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

Epoch [1/10000], Loss: 18.1124
Epoch [501/10000], Loss: 16.3010
Epoch [1001/10000], Loss: 18.5695
Epoch [1501/10000], Loss: 17.9249
Epoch [2001/10000], Loss: 17.1661
Epoch [2501/10000], Loss: 18.1607
Epoch [3001/10000], Loss: 14.4619
Epoch [3501/10000], Loss: 14.8445
Epoch [4001/10000], Loss: 22.8916
Epoch [4501/10000], Loss: 18.9038
Epoch [5001/10000], Loss: 18.1075
Epoch [5501/10000], Loss: 16.6824
Epoch [6001/10000], Loss: 5.9086
Epoch [6501/10000], Loss: 3.6089
Epoch [7001/10000], Loss: 2.5379
Epoch [7501/10000], Loss: 3.0338
Epoch [8001/10000], Loss: 1.9298
Epoch [8501/10000], Loss: 1.8529
Epoch [9001/10000], Loss: 1.8207
Epoch [9501/10000], Loss: 1.9277
Epoch [10000/10000], Loss: 1.2841


In [49]:
model.eval()
with torch.no_grad():
    total_loss = 0.0
    for inputs, labels in test_loader:
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        total_loss += loss.item()
        
    avg_loss = total_loss / len(test_loader)
    print(f'Average Loss of the model on the test set: {avg_loss:.4f}')

Average Loss of the model on the test set: 2.0985


In [50]:
# Save the model state dictionary
torch.save(model.state_dict(), 'mlp_model.pth')
print('Model saved to mlp_model.pth')

Model saved to mlp_model.pth


# Inference again

In [3]:
# Load the data from a CSV file
data = pd.read_csv('hand_landmark_data.csv')

X = data.iloc[:, :-63].values

In [4]:
# Load the model state dictionary
model = MLP(X.shape[1], 64, 63)
model.load_state_dict(torch.load('mlp_model.pth'))
model.eval()  # Set the model to evaluation mode
print('Model loaded from mlp_model.pth')

Model loaded from mlp_model.pth


In [5]:
X = torch.tensor(X, dtype=torch.float32)

In [6]:
Y = model(X)

In [7]:
hand_lnmks = Y[-1]

In [8]:
hand_lnmks = hand_lnmks.reshape(21, 3)

In [9]:
hand_lnmks = hand_lnmks.detach().numpy()

In [10]:
# Create the initial point cloud
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(hand_lnmks)

lines = [[0,1],[1,2],[2,3],[3,4], 
         [0,5],[5,6],[6,7],[7,8],
         [5,9],[9,10],[10,11],[11,12],
         [9,13],[13,14],[14,15],[15,16],
         [13,17],[17,18],[18,19],[19,20],[0,17]]
colors = [[1, 0, 0] for i in range(len(lines))]
line_set = o3d.geometry.LineSet(
    points=o3d.utility.Vector3dVector(hand_lnmks),
    lines=o3d.utility.Vector2iVector(lines),
)
line_set.colors = o3d.utility.Vector3dVector(colors)

In [11]:
o3d.visualization.draw_geometries([pcd, line_set])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


# Visualize GTs

In [3]:
hand_lmks_file = np.load('../hand_landmarks.npz')
hand_lmks_gt = hand_lmks_file["landmarks"]

In [ ]:
x = np.array([[500, 0, 0],
              [0, 0, 0]])
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(x)

lines = [[0, 0]]
colors = [[1, 0, 0] for i in range(len(lines))]
line_set = o3d.geometry.LineSet(
    points=o3d.utility.Vector3dVector(x),
    lines=o3d.utility.Vector2iVector(lines)
)
line_set.colors = o3d.utility.Vector3dVector(colors)
    
vis = o3d.visualization.Visualizer()
vis.create_window()
vis.add_geometry(pcd)
vis.add_geometry(line_set)

i = 0
while True:
    hand_lmks = hand_lmks_gt[i]
    pcd.points = o3d.utility.Vector3dVector(hand_lmks)

    lines = [[0,1],[1,2],[2,3],[3,4], 
             [0,5],[5,6],[6,7],[7,8],
             [5,9],[9,10],[10,11],[11,12],
             [9,13],[13,14],[14,15],[15,16],
             [13,17],[17,18],[18,19],[19,20],[0,17]]
    colors = [[1, 0, 0] for i in range(len(lines))]
    line_set.points = o3d.utility.Vector3dVector(hand_lmks)  # Update the points
    line_set.lines = o3d.utility.Vector2iVector(lines)  # Update the lines
    line_set.colors = o3d.utility.Vector3dVector(colors)

    vis.update_geometry(pcd)
    vis.update_geometry(line_set)
    vis.poll_events()
    vis.update_renderer()

    i += 1
    if i == hand_lmks_gt.shape[0]:
        i = 0
        
    time.sleep(0.1)

vis.destroy_window()

# Visualize predictions

In [3]:
# Load the data from a CSV file
data = pd.read_csv('hand_landmark_data.csv')

X = data.iloc[:, :-63].values

In [4]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(device)

cpu


In [5]:
# Load the model state dictionary
model = MLP(X.shape[1], 64, 63).to(device)
model.load_state_dict(torch.load('mlp_model.pth'))
model.eval()  # Set the model to evaluation mode
print('Model loaded from mlp_model.pth')

Model loaded from mlp_model.pth


In [6]:
X = torch.tensor(X, dtype=torch.float32)

In [ ]:
x = np.array([[500, 0, 0],
              [0, 0, 0]])
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(x)

lines = [[0, 0]]
colors = [[1, 0, 0] for i in range(len(lines))]
line_set = o3d.geometry.LineSet(
    points=o3d.utility.Vector3dVector(x),
    lines=o3d.utility.Vector2iVector(lines)
)
line_set.colors = o3d.utility.Vector3dVector(colors)
    
vis = o3d.visualization.Visualizer()
vis.create_window()
vis.add_geometry(pcd)
vis.add_geometry(line_set)

i = 0
while True:
    lmks_input = X[i].to(device)
    lmks_output = model(lmks_input)
    lmks_output = lmks_output.detach().to("cpu").numpy()
    lmks_output = lmks_output.reshape(-1, 3)
    pcd.points = o3d.utility.Vector3dVector(lmks_output)

    lines = [[0,1],[1,2],[2,3],[3,4], 
             [0,5],[5,6],[6,7],[7,8],
             [5,9],[9,10],[10,11],[11,12],
             [9,13],[13,14],[14,15],[15,16],
             [13,17],[17,18],[18,19],[19,20],[0,17]]
    colors = [[1, 0, 0] for i in range(len(lines))]
    line_set.points = o3d.utility.Vector3dVector(lmks_output)  # Update the points
    line_set.lines = o3d.utility.Vector2iVector(lines)  # Update the lines
    line_set.colors = o3d.utility.Vector3dVector(colors)

    vis.update_geometry(pcd)
    vis.update_geometry(line_set)
    vis.poll_events()
    vis.update_renderer()

    i += 1
    if i == X.shape[0]:
        i = 0
        
    time.sleep(0.1)

vis.destroy_window()

In [ ]:
21 * 3 * 2

In [ ]:
21 * 3